# MNIST Action Plan
- Outline the model and choose the activation fn we want to employ
- Describe the placeholder, variables, and the related operations
- Choose the approprate advanced optimizers
- Split the dataset into patches for faster learning
- Initialize the variables
- Make it learn
- Test the accuracy of the model.

In [2]:
#Import Relevant packages
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

In [8]:
# Outline the model
input_size = 784
output_size = 10
hidden_layer_size = 100

tf.reset_default_graph() #clear the memory all variable left form prev. run

inputs = tf.placeholder(tf.float32,[None,input_size])
targets = tf.placeholder(tf.float32,[None,output_size])

#tf.nn contain most of activation fn
weights_1 = tf.get_variable("weight_1",[input_size, hidden_layer_size])
biases_1 = tf.get_variable("biases_1",[hidden_layer_size])
outputs_1 = tf.nn.relu(tf.matmul(inputs,weights_1) + biases_1)

weights_2 = tf.get_variable("weight_2",[hidden_layer_size, hidden_layer_size])
biases_2 = tf.get_variable("biases_2",[hidden_layer_size])
outputs_2 = tf.nn.relu(tf.matmul(outputs_1,weights_2) + biases_2)

weights_3 = tf.get_variable("weight_3",[hidden_layer_size, output_size])
biases_3 = tf.get_variable("biases_3",[output_size])

outputs = tf.matmul(outputs_2,weights_3) + biases_3
#logits = output, labels=targets
loss = tf.nn.softmax_cross_entropy_with_logits(logits=outputs, labels=targets)
#reduce_mean is a method which finds the mean of tensor across dimension
mean_loss = tf.reduce_mean(loss)
#optimizer
#Also optimize = tf.train.GradientDescentOptimizer(learning_rate=0.05).minimize(mean_loss)
optimize = tf.train.AdamOptimizer(learning_rate=0.001).minimize(mean_loss)

#contain 1 or 0 for each observation
out_equal_target = tf.equal(tf.argmax(outputs,1), tf.argmax(targets,1)) #argmax 1 i.e,(column) cause we are interested in max arg in target

#tf.cast method to cast an obj to another datatype
accuracy = tf.reduce_mean(tf.cast(out_equal_target, tf.float32))

sess = tf.InteractiveSession()
initializer = tf.global_variables_initializer()
sess.run(initializer)

#batch_size of 1 = SGD, batch_size no. of sample = GD
#batches = No. of samples/batch size
batch_size = 100
baches = mnist.train._num_examples // batch_size
max_epochs = 15

# the true early stop will come if the validation loss starts increasing
prev_validation_loss = 9999999.

#we will go to every epochs we will go to saveral batches and update the weights multiple time instead of one per epochs
for epoch_counter in range(max_epochs):
    curr_epoch_loss = 0. #contains the mean training loss
    # we wont lose any info as everythng is backpropagated to the parameter 
    for batch_counter in range(baches): #it loads 100 i/p and 100 o/p as batch size=100
        #next_batch load the batches one after other (comes with mnist)
        input_batch, target_batch = mnist.train.next_batch(batch_size)
        #session run operation
        #optimizes the algo and calculates the batch loss
        _, batch_loss = sess.run([optimize, mean_loss],
                                feed_dict = {inputs: input_batch, targets :target_batch})
        #record loss for the iteration 
        curr_epoch_loss += batch_loss #now repeation till end
    
    #Avg loss or mean loss
    curr_epoch_loss /= baches
    
    #now validation loss and accuracy for validation dataset
    input_batch, target_batch = mnist.validation.next_batch(mnist.validation._num_examples)
    validation_loss, validation_accuracy = sess.run([mean_loss, accuracy], 
                                                    feed_dict = {inputs: input_batch, targets :target_batch})
    
    print('Epochs '+str(epoch_counter+1)+
         '. Training Loss : '+'{0:.3f}'.format(curr_epoch_loss)+
         '. Validation Loss : '+'{0:.3f}'.format(validation_loss)+
         '. Validation Accuracy : '+'{0:.2f}'.format(validation_accuracy * 100.)+'%')
    
    if validation_loss > prev_validation_loss:
        break
    prev_validation_loss = validation_loss
    
print('End of Training')

C:\Users\codehax41\Anaconda3\lib\site-packages\tensorflow\python\client\session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Epochs 1. Training Loss : 0.338. Validation Loss : 0.169. Validation Accuracy : 95.30%
Epochs 2. Training Loss : 0.134. Validation Loss : 0.117. Validation Accuracy : 96.50%
Epochs 3. Training Loss : 0.095. Validation Loss : 0.096. Validation Accuracy : 97.18%
Epochs 4. Training Loss : 0.072. Validation Loss : 0.086. Validation Accuracy : 97.46%
Epochs 5. Training Loss : 0.056. Validation Loss : 0.076. Validation Accuracy : 97.66%
Epochs 6. Training Loss : 0.046. Validation Loss : 0.074. Validation Accuracy : 97.76%
Epochs 7. Training Loss : 0.040. Validation Loss : 0.082. Validation Accuracy : 97.48%
End of Training


#### Now on test dataset

In [9]:
input_batch, target_batch = mnist.test.next_batch(mnist.test._num_examples)
test_accuracy = sess.run([accuracy],
                        feed_dict={inputs : input_batch, targets: target_batch})
test_accuracy_percent = test_accuracy[0]*100.
print('Test accuracy: '+'{0:.2f}'.format(test_accuracy_percent)+'%')

Test accuracy: 97.51%
